<a href="https://colab.research.google.com/github/giordamaug/BIONETdatasets/blob/main/TUD/notebooks/NetWL2Vec example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/giordamaug/BIONETdatasets/main?filepath=TUD%2Fnotebooks%2FNetWL2Vec+example.ipynb)
<a href="https://kaggle.com/kernels/welcome?src=https://github.com/giordamaug/BIONETdatasets/blob/main/TUD/notebooks/NetWL2Vec example.ipynb" target="_parent"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Kaggle"/></a>

# Cloning BIONETdatasets repository

In [ ]:
!git clone http://github.com/giordamaug/BIONETdatasets

# Install Netpro2vec and python-igraph (if required)

In [1]:
# Install python-igraph (if required)
!pip install -q git+https://github.com/cds-group/Netpro2vec.git
!pip install -q python-igraph

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


# Load the dataset

In [31]:
dataname = 'MUTAG'
import sys, os
sys.path.append(os.path.abspath('BIONETdatasets/TUD'))
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../../methods'))
from wrappers.igraph_wrapper import MyTUDataset
import shutil
shutil.unpack_archive(f'../datasets/{dataname}.zip')
#shutil.unpack_archive(f'BIONETdatasets/TUD/datasets/{dataname}.zip')
dataset = MyTUDataset(dataname, path=f'{dataname}', verbose=True)
print(dataset.graph_list[0])

Loading edges:: 100%|██████████| 7442/7442 [00:00<00:00, 107391.22it/s]

Loading node labels...
Loading edge labels...
Loading graph labels...
IGRAPH UN-- 17 19 -- MUTAG_1
+ attr: class (g), name (g), label (v), name (v), label (e)
+ edges (vertex names):
0--1, 1--2, 2--3, 3--4, 4--5, 0--5, 4--6, 6--7, 7--8, 8--9, 3--9, 9--10,
10--11, 11--12, 12--13, 8--13, 12--14, 14--15, 14--16


# Create model and evaluate

In [83]:
dataname = 'MUTAG'
import sys, os
sys.path.append(os.path.abspath('BIONETdatasets/TUD'))
sys.path.append(os.path.abspath('..'))
from wrappers.networkx_wrapper import MyTUDataset
import shutil
shutil.unpack_archive(f'../datasets/{dataname}.zip')
#shutil.unpack_archive(f'BIONETdatasets/TUD/datasets/{dataname}.zip')
dataset = MyTUDataset(dataname, path=f'{dataname}', verbose=True)
print(dataset.graph_list[0])

Loading edges:: 100%|██████████| 7442/7442 [00:00<00:00, 38823.54it/s]

Loading node labels...
Loading edge labels...
Loading graph labels...
Graph named 'MUTAG_1' with 17 nodes and 19 edges


In [93]:
import sys, os
import numpy as np
sys.path.append(os.path.abspath('BIONETdatasets/methods'))
if 'netwld2v.Netwld2v' in sys.modules:  
    print("deleting")
    del sys.modules["netwld2v.Netwld2v"]
from netwld2v.Netwld2v import Netwld2v
from netwld2v.graph2vec import Graph2Vec

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from wrappers.igraph_wrapper import MyTUDataset

# Set optimal params
#params = {"annotation" : 'degree', "wl_iterations" : 2, "dimensions": 128, "epochs": 100, "min_count": 2,  "seed": 1, "verbose": True, "vertex_attribute": 'label', "workers": 4}
params = {"wl_iterations" : 3, "dimensions": 128, "epochs": 100, "min_count": 2,  "seed": 1, "workers": 4}
# Transductive embedding
print("TRANSDUCTIVE EMBEDDING:")
model = Graph2Vec(**params)
model.fit(dataset.graph_list)
X = model.get_embedding()
y = dataset.graph_labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
clf = SVC(kernel='linear')
preds = clf.fit(X_train,y_train).predict(X_test)
print(confusion_matrix(y_test, preds),f'Acc. {accuracy_score(y_test, preds)}')
# Inductive embedding
print(len(model.get_documents()[1].words))
model.get_documents()[1].words

deleting
TRANSDUCTIVE EMBEDDING:
[[ 4  3]
 [ 0 12]] Acc. 0.8421052631578947
52


['4',
 '0cce1d019f4dfe3f7e302db488dffe57',
 'dc882840a7cc3789e8107750e2efdc36',
 '6d431a0a91429563d630b5d8fadc4e59',
 '4',
 'f4490329afa6428ebf9afd9005f27abf',
 'b3a2337249583f88e793adf89f051b5c',
 'a8a30605b677696f9d8124c1bba3f9fe',
 '5',
 'daf0ea2428a35efce0f888355b911b7b',
 'c8df4cac549023c512764bc498ec5fe1',
 'b09a2538ebe67d2036707e8f4002b786',
 '4',
 'f4490329afa6428ebf9afd9005f27abf',
 'b3a2337249583f88e793adf89f051b5c',
 '2e47138c1ce882be5e90e002f2e64db3',
 '4',
 '0cce1d019f4dfe3f7e302db488dffe57',
 'd2f28a9d851e79d522463d5b5186f3f8',
 '381773c161d0afc2aa6b4f00b39af7b2',
 '4',
 '0cce1d019f4dfe3f7e302db488dffe57',
 'd2f28a9d851e79d522463d5b5186f3f8',
 'a070aad4ed720ef1d1359a8dd7f6b15d',
 '4',
 'f4490329afa6428ebf9afd9005f27abf',
 'cd91549c92600d50c643cc771e36d017',
 '3a7ab5ecf88b5287167d95e63af22ac9',
 '5',
 '101640cf30a7e8aa5631df7b2fe6afba',
 '0671e6c7d11e6473200d57a7192db7d0',
 'f11d533f30c9139270b3c85eb3981273',
 '5',
 '101640cf30a7e8aa5631df7b2fe6afba',
 '05839a0844503a4b077